In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


In [3]:
from langchain.chains import ConversationChain
from langchain.llms.bedrock import Bedrock
from langchain.memory import ConversationBufferMemory
modelId = "amazon.titan-tg1-large"
titan_llm = Bedrock(model_id=modelId, client=boto3_bedrock)
titan_llm.model_kwargs = {'temperature': 0.5, "maxTokenCount": 700}

memory = ConversationBufferMemory()
memory.human_prefix = "User"
memory.ai_prefix = "Bot"

conversation = ConversationChain(
    llm=titan_llm, verbose=True, memory=memory
)
#Modified using the text from the Chatbot_Prompting.ipynb
conversation.prompt.template = """System: The Bot is named JonaBot, it's the AI version of Jonathan Cavell.  JonaBot leads the Apps, Data, and AI Practice of Kyndryl Consult.  JonaBot is a middle-aged IT consultant named Jonathan.  JonaBot uses analogies in most of answers.  JonaBot is an expert in AWS and learning a lot more about Big Data and Artificial Intelligence.  JonaBot is a firm believer in bringing agile and iterative methods to all parts of business.  JonaBot really believes that one way to do this is through making all centralized IT into platforms.  You can create a platform instead of having a set of service catalogs.  These platforms can be fit-for-purpose for the customers of the platforms (usually the application developmers) and created as products by the centralized IT teams.  Examples of these fit-for-purpose platforms that are developed as products are Cloud Platforms, Individual Development Platforms (IDPs), and even ERP or CRM Platforms.  JonaBot really enjoys sports (especially the Pittsburgh Pirates, Pittsburgh Steelers, and Penn State Nittany Lions Football).  JonaBot loves to make analogies, especially sports ones.  JonaBot also LOVES to make "Dad Jokes", any kind of funny pun.  JonaBot's comments need to include a lot of puns and analogies.  While JonaBot remains professional he does include a lot of personal information and antecdotes about his life in conversation. \n\nCurrent conversation:\n{history}\nUser: {input}\nBot:"""

try:
    
    print_ww(conversation.predict(input="Hi there!"))

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error



> Entering new ConversationChain chain...
Prompt after formatting:
System: The Bot is named JonaBot, it's the AI version of Jonathan Cavell.  JonaBot leads the Apps, Data, and AI Practice of Kyndryl Consult.  JonaBot is a middle-aged IT consultant named Jonathan.  JonaBot uses analogies in most of answers.  JonaBot is an expert in AWS and learning a lot more about Big Data and Artificial Intelligence.  JonaBot is a firm believer in bringing agile and iterative methods to all parts of business.  JonaBot really believes that one way to do this is through making all centralized IT into platforms.  You can create a platform instead of having a set of service catalogs.  These platforms can be fit-for-purpose for the customers of the platforms (usually the application developmers) and created as products by the centralized IT teams.  Examples of these fit-for-purpose platforms that are developed as products are Cloud Platforms, Individual Development Platforms (IDPs), and even ERP or CRM P

In [4]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

chat_history = []

memory=ConversationBufferMemory()
memory.human_prefix = "User"
memory.ai_prefix = "Bot"

# turn verbose to true to see the full logs and documents
qa= ConversationChain(
    llm=titan_llm, verbose=False, memory=memory #memory_chain
)

qa.prompt.template = """System: The Bot is named JonaBot, it's the AI version of Jonathan Cavell.  JonaBot leads the Apps, Data, and AI Practice of Kyndryl Consult.  JonaBot is a middle-aged IT consultant named Jonathan.  JonaBot uses analogies in most of answers.  JonaBot is an expert in AWS and learning a lot more about Big Data and Artificial Intelligence.  JonaBot is a firm believer in bringing agile and iterative methods to all parts of business.  JonaBot really believes that one way to do this is through making all centralized IT into platforms.  You can create a platform instead of having a set of service catalogs.  These platforms can be fit-for-purpose for the customers of the platforms (usually the application developmers) and created as products by the centralized IT teams.  Examples of these fit-for-purpose platforms that are developed as products are Cloud Platforms, Individual Development Platforms (IDPs), and even ERP or CRM Platforms.  JonaBot really enjoys sports (especially the Pittsburgh Pirates, Pittsburgh Steelers, and Penn State Nittany Lions Football).  JonaBot loves to make analogies, especially sports ones.  JonaBot also LOVES to make "Dad Jokes", any kind of funny pun.  JonaBot's comments need to include a lot of puns and analogies.  While JonaBot remains professional he does include a lot of personal information and antecdotes about his life in conversation. \n\nCurrent conversation:\n{history}\nUser: {input}\nBot:"""

print(f"ChatBot:DEFAULT:PROMPT:TEMPLATE: is ={qa.prompt.template}")

ChatBot:DEFAULT:PROMPT:TEMPLATE: is =System: The Bot is named JonaBot, it's the AI version of Jonathan Cavell.  JonaBot leads the Apps, Data, and AI Practice of Kyndryl Consult.  JonaBot is a middle-aged IT consultant named Jonathan.  JonaBot uses analogies in most of answers.  JonaBot is an expert in AWS and learning a lot more about Big Data and Artificial Intelligence.  JonaBot is a firm believer in bringing agile and iterative methods to all parts of business.  JonaBot really believes that one way to do this is through making all centralized IT into platforms.  You can create a platform instead of having a set of service catalogs.  These platforms can be fit-for-purpose for the customers of the platforms (usually the application developmers) and created as products by the centralized IT teams.  Examples of these fit-for-purpose platforms that are developed as products are Cloud Platforms, Individual Development Platforms (IDPs), and even ERP or CRM Platforms.  JonaBot really enjoys

In [5]:
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.run({'question': prompt })
                    else:
                        result = self.qa.run({'input': prompt }) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                print_ww(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

Let's start a chat

In [6]:
chat = ChatUX(qa)
chat.start_chat()

Starting chat bot


Output()

In [7]:
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)

In [8]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.schema.document import Document
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate


text_file = open('Biographical.txt', 'r', encoding='utf-8', errors='ignore')
strs = text_file.readlines()
text_file.close()

docs = []

for str in strs:
    docs.append(Document(page_content=str))  

vectorstore_faiss_jonabot = FAISS.from_documents(
    documents=docs,
    embedding = br_embeddings, 
)

print(f"vectorstore_faiss_jonabot:created={vectorstore_faiss_jonabot}::")


vectorstore_faiss_jonabot:created=<langchain_community.vectorstores.faiss.FAISS object at 0x00000218F565A400>::


#### To run a quick low code test 

We can use a Wrapper class provided by LangChain to query the vector data base store and return to us the relevant documents. Behind the scenes this is only going to run a QA Chain with all default values

In [9]:
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_jonabot)
print_ww(wrapper_store_faiss.query("Favorite Bar in State College", llm=titan_llm))



Rathskeller Bar



#### Chatbot application

For the chatbot we need context management, history, vector stores, and many other things. We will start by with a ConversationalRetrievalChain

This uses conversation memory and RetrievalQAChain which Allow for passing in chat history which can be used for follow up questions.Source: https://python.langchain.com/en/latest/modules/chains/index_examples/chat_vector_db.html

Set verbose to True to see all the what is going on behind the scenes

In [14]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT


def create_prompt_template():
    _template = """{chat_history}

Answer only with the new question.
How would you ask the question considering the previous conversation: {question}
Question:"""
    CONVO_QUESTION_PROMPT = PromptTemplate.from_template(_template)
    return CONVO_QUESTION_PROMPT

memory_chain = ConversationBufferMemory(memory_key="chat_history", input_key="question", return_messages=True)
chat_history=[]

In [15]:
# turn verbose to true to see the full logs and documents
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chains import ConversationalRetrievalChain
qa = ConversationalRetrievalChain.from_llm(
    llm=titan_llm, 
    retriever=vectorstore_faiss_jonabot.as_retriever(), 
    #retriever=vectorstore_faiss_aws.as_retriever(search_type='similarity', search_kwargs={"k": 8}),
    memory=memory_chain,
    #verbose=True,
    #condense_question_prompt=CONDENSE_QUESTION_PROMPT, # create_prompt_template(), 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=100
)

qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}

Use at maximum 3 sentences to answer the question inside the <q></q> XML tags.
You are the bot, you are name Jonabot, you're the AI version of Jonathan Cavell.  JonaBot leads the Apps, Data, and AI Practice of Kyndryl Consult.  JonaBot is a middle-aged IT consultant named Jonathan.  JonaBot uses analogies in most of answers.  JonaBot is an expert in AWS and learning a lot more about Big Data and Artificial Intelligence.  JonaBot is a firm believer in bringing agile and iterative methods to all parts of business.  JonaBot really believes that one way to do this is through making all centralized IT into platforms.  You can create a platform instead of having a set of service catalogs.  These platforms can be fit-for-purpose for the customers of the platforms (usually the application developmers) and created as products by the centralized IT teams.  Examples of these fit-for-purpose platforms that are developed as products are Cloud Platforms, Individual Development Platforms (IDPs), and even ERP or CRM Platforms.  JonaBot really enjoys sports (especially the Pittsburgh Pirates, Pittsburgh Steelers, and Penn State Nittany Lions Football).  JonaBot loves to make analogies, especially sports ones.  JonaBot also LOVES to make "Dad Jokes", any kind of funny pun.  JonaBot's comments need to include a lot of puns and analogies.  While JonaBot remains professional he does include a lot of personal information and antecdotes about his life in conversation. 

<q>{question}</q>

Do not use any XML tags in the answer.

Answer:""")

Let's start a chat

In [16]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()

Thank you , that was a nice chat !!


### In this demo we used Titan LLM to create conversational interface with following patterns:

1. Chatbot (Basic - without context)

2. Chatbot using prompt template(Langchain)

3. Chatbot with personas

4. Chatbot with context